In [3]:
# Импорт нужной для работы библиотеки
import win32com.client
rastr = win32com.client.Dispatch('Astra.Rastr')

import data_extract

In [4]:
import sys

# Если данная ячейка завершается ошибкой,
# значит вы используете не 32-битную версию Python
assert sys.maxsize < 2**32, \
    'Incorrect Python architecture. We should use Python 32-bit'

In [5]:
# Загружаем режим
rastr.Load(1, 'regime.rg2', '')
result = rastr.rgm('p')
print(result)

0


In [4]:
# Парсим json с сечением

flowgate_lines = data_extract.json_to_dic('flowgate.json')

print(flowgate_lines)

#with open('flowgate.json', "r") as flowgate:
#    flowgate_data = json.load(flowgate)

#print(flowgate_add('flowgate.json'))

#for line in flowgate_data:
#    print(flowgate_data[line]['ip'])

{'line_1': {'ip': 17, 'iq': 16, 'np': 0}, 'line_2': {'ip': 6, 'iq': 11, 'np': 0}, 'line_3': {'ip': 4, 'iq': 14, 'np': 0}}


In [5]:
# Парсим json с отключениями

faults_lines = data_extract.json_to_dic('faults.json')

print(faults_lines)

{'outage_of_6_11': {'ip': 6, 'iq': 11, 'np': 0, 'sta': 1}, 'outage_of_4_14': {'ip': 4, 'iq': 14, 'np': 0, 'sta': 1}}


In [6]:
# Парсим csv с траекторией

trajectory_nodes = data_extract.csv_to_list('vector.csv')

print(trajectory_nodes)

[{'variable': 'pn', 'node': '23', 'value': '-3.0', 'tg': '1'}, {'variable': 'pn', 'node': '24', 'value': '-3.0', 'tg': '1'}, {'variable': 'pn', 'node': '29', 'value': '1.0', 'tg': '1'}, {'variable': 'pn', 'node': '21', 'value': '-3.0', 'tg': '1'}, {'variable': 'pn', 'node': '27', 'value': '2.0', 'tg': '1'}, {'variable': 'pn', 'node': '3', 'value': '2.0', 'tg': '1'}, {'variable': 'pn', 'node': '16', 'value': '-4.0', 'tg': '1'}, {'variable': 'pn', 'node': '8', 'value': '3.0', 'tg': '1'}, {'variable': 'pn', 'node': '7', 'value': '2.0', 'tg': '1'}, {'variable': 'pn', 'node': '15', 'value': '-4.0', 'tg': '1'}, {'variable': 'pn', 'node': '4', 'value': '4.0', 'tg': '1'}, {'variable': 'pn', 'node': '20', 'value': '-1.0', 'tg': '0'}, {'variable': 'pn', 'node': '25', 'value': '4.0', 'tg': '0'}, {'variable': 'pg', 'node': '1', 'value': '4.0', 'tg': '0'}, {'variable': 'pg', 'node': '2', 'value': '3.0', 'tg': '0'}, {'variable': 'pg', 'node': '3', 'value': '3.0', 'tg': '0'}, {'variable': 'pg', 'node

In [7]:
#Словарик узлов
node_data = {}
i = 0
for node in trajectory_data:
    if node['node'] not in node_data:
        node_data[node['node']] = i
    i = i + 1
print(node_data)

{'23': 0, '24': 1, '29': 2, '21': 3, '27': 4, '3': 5, '16': 6, '8': 7, '7': 8, '15': 9, '4': 10, '20': 11, '25': 12, '1': 13, '2': 14, '5': 17, '6': 18, '9': 21}


In [8]:
# Формируем и загружаем шаблоны
rastr.Save('sech.sch', 'C:/Users/mishk/Documents/RastrWin3/SHABLON/сечения.sch')
rastr.Load(1, 'sech.sch', 'C:/Users/mishk/Documents/RastrWin3/SHABLON/сечения.sch')

rastr.Save('traj.ut2', 'C:/Users/mishk/Documents/RastrWin3/SHABLON/траектория утяжеления.ut2')
rastr.Load(1, 'traj.ut2', 'C:/Users/mishk/Documents/RastrWin3/SHABLON/траектория утяжеления.ut2')

In [9]:
#Заполняем sech.sch
FlowGate = rastr.Tables('sechen')
GroupLine = rastr.Tables('grline')

FlowGate.DelRowS()
GroupLine.DelRows()

FlowGate.AddRow()
FlowGate.Cols('name').SetZ(0, 'new')
FlowGate.Cols('ns').SetZ(0, 1)
FlowGate.Cols('sta').SetZ(0, 1)

i = 0

for line in flowgate_data:
    GroupLine.AddRow()
    GroupLine.Cols('ns').SetZ(i, 1)

    start_node = flowgate_data[line]['ip']
    end_node = flowgate_data[line]['iq']

    GroupLine.Cols('ip').SetZ(i, start_node)
    GroupLine.Cols('iq').SetZ(i, end_node)
    
    i = i + 1

rastr.Save('sech.sch', 'C:/Users/mishk/Documents/RastrWin3/SHABLON/сечения.sch')

In [10]:
#Заполняем traj.ut2
Trajectory = rastr.Tables('ut_node')

i = 0
node_dic = {}

Trajectory.DelRowS()

for node in trajectory_data:
    node_type = node['variable']
    node_number = node['node']
    power_change = float(node['value'])
    power_tg = node['tg']

    if node_number not in node_dic:
        node_dic[node_number] = i
        i = i + 1
        Trajectory.AddRow()
        Trajectory.Cols('ny').SetZ(node_dic[node_number], node_number)
        Trajectory.Cols(node_type).SetZ(node_dic[node_number], power_change)
    else:
        Trajectory.Cols(node_type).SetZ(node_dic[node_number], power_change)

    if Trajectory.Cols('tg').Z(node_dic[node_number]) == 0:
        Trajectory.Cols('tg').SetZ(node_dic[node_number], power_tg)

rastr.Save('traj.ut2', 'C:/Users/mishk/Documents/RastrWin3/SHABLON/траектория утяжеления.ut2')

In [11]:
# Тестовое утяжеление
rastr.Load(1, 'regime.rg2', 'C:/Users/mishk/Documents/RastrWin3/SHABLON/режим.rg2')
rastr.Load(1, 'sech.sch', 'C:/Users/mishk/Documents/RastrWin3/SHABLON/сечения.sch')
rastr.Load(1, 'traj.ut2', 'C:/Users/mishk/Documents/RastrWin3/SHABLON/траектория утяжеления.ut2')

result = rastr.rgm('p')
print(result)

0


In [12]:
# Параметры утяжеления
PowerFlowControl = rastr.Tables('ut_common')
PowerFlowControl.Cols('iter').SetZ(0, 200)
PowerFlowControl.Cols('enable_contr').SetZ(0, 1)
PowerFlowControl.Cols('dis_v_contr').SetZ(0, 1)
PowerFlowControl.Cols('dis_i_contr').SetZ(0, 1)

In [13]:
#Пробное утяжеление
if rastr.ut_utr('i') > 0:
    rastr.ut_utr('')

maxPowerFlow = round(abs(rastr.Tables('sechen').Cols('psech').Z(0)), 2)
print(maxPowerFlow)

2808.21


In [7]:
#Обеспечение нормативного коэффициента запаса статической апериодической устойчивости 
# по активной мощности в контролируемом сечении в нормальной схеме

rastr.Load(1, 'regime.rg2', 'C:/Users/mishk/Documents/RastrWin3/SHABLON/режим.rg2')
rastr.Load(1, 'sech.sch', 'C:/Users/mishk/Documents/RastrWin3/SHABLON/сечения.sch')
rastr.Load(1, 'traj.ut2', 'C:/Users/mishk/Documents/RastrWin3/SHABLON/траектория утяжеления.ut2')

PowerFlowControl = rastr.Tables('ut_common')
PowerFlowControl.Cols('iter').SetZ(0, 200)
PowerFlowControl.Cols('enable_contr').SetZ(0, 1)
PowerFlowControl.Cols('dis_v_contr').SetZ(0, 1)
PowerFlowControl.Cols('dis_i_contr').SetZ(0, 1)

#print(type(PowerFlowControl.Cols('enable_contr').Z(0))

if rastr.ut_utr('i') > 0:
    rastr.ut_utr('')

Criteria1 = abs(rastr.Tables('sechen').Cols('psech').Z(0)) * 0.8 - 30
Criteria1 = round(Criteria1, 2)
print(Criteria1)

2216.57


In [50]:
#Обеспечение нормативного коэффициента запаса статической устойчивости
# по напряжению в узлах нагрузки в нормальной схеме

rastr.Load(1, 'regime.rg2', 'C:/Users/mishk/Documents/RastrWin3/SHABLON/режим.rg2')
rastr.Load(1, 'sech.sch', 'C:/Users/mishk/Documents/RastrWin3/SHABLON/сечения.sch')
rastr.Load(1, 'traj.ut2', 'C:/Users/mishk/Documents/RastrWin3/SHABLON/траектория утяжеления.ut2')

PowerFlowControl = rastr.Tables('ut_common')
PowerFlowControl.Cols('iter').SetZ(0, 200)
PowerFlowControl.Cols('enable_contr').SetZ(0, 1)
PowerFlowControl.Cols('dis_i_contr').SetZ(0, 1)

i = 0

while i < rastr.Tables('node').Size:
    if rastr.Tables('node').Cols('tip').Z(i) == 1:
        u_kr = rastr.Tables('node').Cols('uhom').Z(i) * 0,7
        u_min = u_kr * 1,15
        rastr.Tables('node').Cols('umin').SetZ(i, u_min)
        rastr.Tables('node').Cols('contr_v').SetZ(i, 1)
    i = i + 1

if rastr.ut_utr('i') > 0:
    rastr.ut_utr('')

#print(abs(rastr.Tables('sechen').Cols('psech').Z(0)))

Criteria2 = abs(rastr.Tables('sechen').Cols('psech').Z(0)) - 30
Criteria2 = round(Criteria2, 2)
print(Criteria2)

#rastr.GetToggle().MoveOnPosition(len(rastr.GetToggle().GetPositions()) - 1)

#abs(rastr.Tables('sechen').Cols('psech').Z(0))

2778.21


In [33]:
print(faults_data)

{'outage_of_6_11': {'ip': 6, 'iq': 11, 'np': 0, 'sta': 1}, 'outage_of_4_14': {'ip': 4, 'iq': 14, 'np': 0, 'sta': 1}}


In [41]:
# Обеспечение нормативного коэффициента запаса статической апериодической устойчивости
# по активной мощности в контролируемом сечении в послеаварийных режимах после нормативных возмущений

rastr.Load(1, 'regime.rg2', 'C:/Users/mishk/Documents/RastrWin3/SHABLON/режим.rg2')
rastr.Load(1, 'sech.sch', 'C:/Users/mishk/Documents/RastrWin3/SHABLON/сечения.sch')
rastr.Load(1, 'traj.ut2', 'C:/Users/mishk/Documents/RastrWin3/SHABLON/траектория утяжеления.ut2')

PowerFlowControl = rastr.Tables('ut_common')
PowerFlowControl.Cols('iter').SetZ(0, 200)
PowerFlowControl.Cols('enable_contr').SetZ(0, 1)
PowerFlowControl.Cols('dis_i_contr').SetZ(0, 1)
PowerFlowControl.Cols('dis_v_contr').SetZ(0, 1)

Critereria3 = []
i = 0
PowerFlow = 0
j = 1

for lines in faults_data:
    while i < rastr.Tables('vetv').Size:
        if (rastr.Tables('vetv').Cols('iq').Z(i) == faults_data[lines]['iq']) and (rastr.Tables('vetv').Cols('ip').Z(i) == faults_data[lines]['ip']):
            
            rastr.Tables('vetv').Cols('sta').SetZ(i, 1)
        
            if rastr.ut_utr('i') > 0:
                rastr.ut_utr('')          
            
            PowerFlow = abs(rastr.Tables('sechen').Cols('psech').Z(0))
            PowerFlow80 = PowerFlow * 0.92

            while PowerFlow > PowerFlow80:
                rastr.GetToggle().MoveOnPosition(len(rastr.GetToggle().GetPositions()) - j)
                PowerFlow = abs(rastr.Tables('sechen').Cols('psech').Z(0))
                j = j + 1
            
            j = 1

            rastr.Tables('vetv').Cols('sta').SetZ(i, 0)
            rastr.rgm('p')

            maxPowerFlow = abs(rastr.Tables('sechen').Cols('psech').Z(0)) - 30
            maxPowerFlow = round(maxPowerFlow, 2)

            Critereria3.append(maxPowerFlow)
            rastr.Load(1, 'regime.rg2', 'C:/Users/mishk/Documents/RastrWin3/SHABLON/режим.rg2')

        i = i + 1
    i = 0    


print(min(Critereria3))

2132.61


In [48]:
# Обеспечение нормативного коэффициента запаса статической устойчивости по напряжению
# в узлах нагрузки в послеаварийных режимах после нормативных возмущений

rastr.Load(1, 'regime.rg2', 'C:/Users/mishk/Documents/RastrWin3/SHABLON/режим.rg2')
rastr.Load(1, 'sech.sch', 'C:/Users/mishk/Documents/RastrWin3/SHABLON/сечения.sch')
rastr.Load(1, 'traj.ut2', 'C:/Users/mishk/Documents/RastrWin3/SHABLON/траектория утяжеления.ut2')

PowerFlowControl = rastr.Tables('ut_common')
PowerFlowControl.Cols('iter').SetZ(0, 200)
PowerFlowControl.Cols('enable_contr').SetZ(0, 1)
PowerFlowControl.Cols('dis_i_contr').SetZ(0, 1)

Critereria4 = []
i = 0
PowerFlow = 0
j = 1
k = 0


while k < rastr.Tables('node').Size:
    if rastr.Tables('node').Cols('tip').Z(k) == 1:
        u_kr = rastr.Tables('node').Cols('uhom').Z(k) * 0,7
        u_min = u_kr * 1,1
        rastr.Tables('node').Cols('umin').SetZ(k, u_min)
    k = k + 1




for lines in faults_data:
    while i < rastr.Tables('vetv').Size:
        if (rastr.Tables('vetv').Cols('iq').Z(i) == faults_data[lines]['iq']) and (rastr.Tables('vetv').Cols('ip').Z(i) == faults_data[lines]['ip']):
            
            rastr.Tables('vetv').Cols('sta').SetZ(i, 1)
        
            if rastr.ut_utr('i') > 0:
                rastr.ut_utr('')          
            
            rastr.Tables('vetv').Cols('sta').SetZ(i, 0)
            rastr.rgm('p')

            maxPowerFlow = abs(rastr.Tables('sechen').Cols('psech').Z(0))
            maxPowerFlow = round(maxPowerFlow, 2)

            Critereria4.append(maxPowerFlow)
            rastr.Load(1, 'regime.rg2', 'C:/Users/mishk/Documents/RastrWin3/SHABLON/режим.rg2')

        i = i + 1
    i = 0    


print(min(Critereria4))

2369.69


In [56]:
#Обеспечение допустимой токовой нагрузки линий электропередачи и электросетевого оборудования в нормальной схеме
rastr.Load(1, 'regime.rg2', 'C:/Users/mishk/Documents/RastrWin3/SHABLON/режим.rg2')
rastr.Load(1, 'sech.sch', 'C:/Users/mishk/Documents/RastrWin3/SHABLON/сечения.sch')
rastr.Load(1, 'traj.ut2', 'C:/Users/mishk/Documents/RastrWin3/SHABLON/траектория утяжеления.ut2')

PowerFlowControl = rastr.Tables('ut_common')
PowerFlowControl.Cols('iter').SetZ(0, 200)
PowerFlowControl.Cols('enable_contr').SetZ(0, 1)
PowerFlowControl.Cols('dis_v_contr').SetZ(0, 1)

j = 1

for line in flowgate_data:
    while i < rastr.Tables('vetv').Size:
        if (flowgate_data[line]['ip'] == rastr.Tables('vetv').Cols('ip').Z(i)) and (flowgate_data[line]['iq'] == rastr.Tables('vetv').Cols('iq').Z(i)):
            rastr.Tables('vetv').Cols('contr_i').SetZ(i, 1)
            rastr.Tables('vetv').Cols('i_dop').SetZ(i, rastr.Tables('vetv').Cols('i_dop_r').Z(i))
        i = i +1
    i = 0

if rastr.ut_utr('i') > 0:
    rastr.ut_utr('')

Criteria5 = abs(rastr.Tables('sechen').Cols('psech').Z(0))
Criteria5 = round(Criteria5, 2)

print(Criteria5)

1817.14


In [60]:
#Обеспечение допустимой токовой нагрузки линий электропередачи и электросетевого оборудования в нормальной схеме
rastr.Load(1, 'regime.rg2', 'C:/Users/mishk/Documents/RastrWin3/SHABLON/режим.rg2')
rastr.Load(1, 'sech.sch', 'C:/Users/mishk/Documents/RastrWin3/SHABLON/сечения.sch')
rastr.Load(1, 'traj.ut2', 'C:/Users/mishk/Documents/RastrWin3/SHABLON/траектория утяжеления.ut2')

PowerFlowControl = rastr.Tables('ut_common')
PowerFlowControl.Cols('iter').SetZ(0, 200)
PowerFlowControl.Cols('enable_contr').SetZ(0, 1)
PowerFlowControl.Cols('dis_v_contr').SetZ(0, 1)

Criteria6 = []
i = 0
j = 0
k = 0

for lines in faults_data:
    
    for line in flowgate_data:
        while i < rastr.Tables('vetv').Size:
            if (flowgate_data[line]['ip'] == rastr.Tables('vetv').Cols('ip').Z(i)) and (flowgate_data[line]['iq'] == rastr.Tables('vetv').Cols('iq').Z(i)):
                rastr.Tables('vetv').Cols('contr_i').SetZ(i, 1)
                rastr.Tables('vetv').Cols('i_dop').SetZ(i, rastr.Tables('vetv').Cols('i_dop_r_av').Z(i))
            i = i + 1
        i = 0
      
    while j < rastr.Tables('vetv').Size:
        if (rastr.Tables('vetv').Cols('iq').Z(j) == faults_data[lines]['iq']) and (rastr.Tables('vetv').Cols('ip').Z(j) == faults_data[lines]['ip']):
            
            rastr.Tables('vetv').Cols('sta').SetZ(j, 1)
        
            if rastr.ut_utr('i') > 0:
                rastr.ut_utr('')          
            
            rastr.Tables('vetv').Cols('sta').SetZ(j, 0)
            rastr.rgm('p')

            maxPowerFlow = abs(rastr.Tables('sechen').Cols('psech').Z(0))
            maxPowerFlow = round(maxPowerFlow, 2)

            Criteria6.append(maxPowerFlow)
            rastr.Load(1, 'regime.rg2', 'C:/Users/mishk/Documents/RastrWin3/SHABLON/режим.rg2')

        j = j + 1
    j = 0

print(min(Criteria6))

1509.26
